## set running directory to project source

In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *
INDY_IP = "192.168.21.6"
INDY_IP2 = "192.168.21.2"

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0.3,-0.4,0), (0,0,np.pi/2)),
                INDY_IP),
    RobotConfig(1, RobotType.indy7, ((-0.3,-0.4,0), (0,0,np.pi/2)),
                INDY_IP2, specs={"no_sdk": True})]
              , connection_list=[False, False])

connection command:
indy0: False
indy1: False


In [3]:
from pkg.detector.aruco.marker_config import *
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.camera.realsense import RealSense
from pkg.detector.camera.kinect import Kinect
aruco_map = get_aruco_map()
stereo = ArucoStereo(aruco_map, [Kinect(), RealSense()])
stereo.initialize()
stereo.calibrate()

Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 5 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 ms. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 ms.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


Start streaming


[(array([[1.82983423e+03, 0.00000000e+00, 1.91572046e+03],
         [0.00000000e+00, 1.82983423e+03, 1.09876086e+03],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([ 7.09966481e-01, -2.73409390e+00,  1.45804870e-03, -3.24774766e-04,
          1.44911301e+00,  5.84310412e-01, -2.56374550e+00,  1.38472950e+00])),
 (array([[1.39560388e+03, 0.00000000e+00, 9.62751587e+02],
         [0.00000000e+00, 1.39531934e+03, 5.47687012e+02],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([0., 0., 0., 0., 0.])),
 array([[ 0.7842825 , -0.0271335 , -0.6198102 ,  0.67792493],
        [ 0.18675563,  0.96302986,  0.19415426, -0.15703547],
        [ 0.59162766, -0.26802483,  0.76035476,  0.08240667],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],
       dtype=float32)]

## create scene builder

In [4]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(stereo)
# deprecated: s_builder.reset_reference_coord(ref_name="floor")

## detect robot and make geometry scene

In [5]:
xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## init planning pipeline

In [6]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## make reference box (gasped)

In [7]:
crob.reset_connection(False, True)

connection command:
indy0: False
indy1: True
Connect: Server IP (192.168.21.2)
   Use a production WSGI server instead.
 * Debug mode: off


In [8]:
rname_calib = "indy1"

In [9]:
tip_link = "indy1_tcp"

In [10]:
robot = crob.robot_dict[rname_calib]

In [11]:
with robot:
    robot.set_joint_vel_level(1)

Connect: Server IP (192.168.21.2)


In [12]:
with robot:
    print(np.round(robot.get_joint_pos(), 4))

Connect: Server IP (192.168.21.2)
[-0.     -0.0002  0.     -0.0001  0.0001 -0.    ]


In [19]:
with robot:
    robot.reset()

Connect: Server IP (192.168.21.2)
Connect: Server IP (192.168.21.2)


In [25]:
with robot:
    Q =robot.get_joint_pos()
print(Q)

Connect: Server IP (192.168.0.106)
[2.2413017336003542, -94.20422262397315, -148.98387057723087, 0.2587225885674675, 163.01584375967968, 3.2264996519185294]


In [26]:
with robot:
    robot.joint_move_to(np.add(Q, [0,0,0,0,-1,0]))

Connect: Server IP (192.168.0.106)


In [20]:
with robot:
    robot.direct_teaching(True)

Connect: Server IP (192.168.21.2)


In [24]:
with robot:
    robot.grasp(True)

Connect: Server IP (192.168.21.2)
Connect: Server IP (192.168.21.2)


In [22]:
with robot:
    robot.reset()

Connect: Server IP (192.168.21.2)
Connect: Server IP (192.168.21.2)


In [25]:
with robot:
    robot.grasp(False)

Connect: Server IP (192.168.21.2)
Connect: Server IP (192.168.21.2)


In [29]:
with robot:
    robot.direct_teaching(False)

Connect: Server IP (192.168.0.106)


In [30]:
with robot:
    robot.go_home()

Connect: Server IP (192.168.0.106)


In [21]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

Please create a subscriber to the marker


In [22]:
bname = gtem_dict.keys()[0]
box = gtem_dict[bname]

In [23]:
Qcur = crob.get_real_robot_pose()

Connect: Server IP (192.168.21.2)


In [24]:
T_be = gscene.get_tf(tip_link, Qcur)
T_bo = box.get_tf(Qcur)
T_eo = np.matmul(SE3_inv(T_be), T_bo)
box.set_link(tip_link)
box.set_offset_tf(center = T_eo[:3,3], orientation_mat=T_eo[:3,:3])
gscene.update_markers_all()

### detect & make data

In [29]:
with robot:
    robot.direct_teaching(True)

Connect: Server IP (192.168.21.2)


In [34]:
data = []

In [35]:
# i_dat = 2
# panda.move_joint_s_curve(data_bak[i_dat][0][6:], N_div=300)

In [36]:
try_mkdir("data/calibdat")

In [58]:
Qcur= crob.get_real_robot_pose()
gscene.show_pose(Qcur)

T_bo = T_xyzrpy(s_builder.detect_items(item_names=[bname])[bname])
T_ref = box.get_tf(list2dict(Qcur, gscene.joint_names))
T_off = np.matmul(SE3_inv(T_ref), T_bo)
rpy_off = Rot2rpy(T_off[:3,:3])
if np.linalg.norm(rpy_off)>0.5:
    print("WARNING: fix orientation - expected offset: {}".format(Rot2rpy(np.round(T_off[:3,:3]))))
data.append((Qcur.copy(), T_bo))
save_pickle("data/calibdat/{:03}.pkl".format(len(data)), data[-1])
len(data)

Connect: Server IP (192.168.21.2)


22

In [90]:
# del data[-1]

In [66]:
with robot:
    robot.direct_teaching(False)

Connect: Server IP (192.168.21.2)


### calibrate

In [62]:
from scipy.optimize import minimize
idx_rob = crob.idx_dict[rname_calib]
box_ref = box
def calc_diff(dq):
    diff_all = []
    for dat in data:
        Qcur = dat[0].copy()
        T_bo = dat[1]
        Qcur[idx_rob] += dq
        T_ref = box_ref.get_tf(list2dict(Qcur, gscene.joint_names))
        T_off = np.matmul(SE3_inv(T_ref), T_bo)
        diff = np.stack([np.matmul(T_off, [0.0,0,0,1])[:3],
                               np.matmul(T_off, [0.025,0,0,1])[:3]-[0.025,0,0],
                               np.matmul(T_off, [0.0,0.025,0,1])[:3]-[0,0.025,0],
                               np.matmul(T_off, [0,0,0.025,1])[:3]-[0,0,0.025]])
        diff_all.append(diff)
    diff_all = np.array(diff_all)
    return np.sum(np.std(diff_all, 0))+np.sum(np.abs(diff_all[:,0, :]))*0.01+np.sum(np.abs(dq))*0.01

In [63]:
x0 = np.zeros(6)

In [64]:
res = minimize(calc_diff, x0, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})
print("initial value: {}".format(calc_diff(x0)))
print("final value: {}".format(calc_diff(res.x)))
print("mean initial value: \t \t {}".format(calc_diff([0]*6)/len(data)/4))
print("mean final value: \t \t {}".format(calc_diff(res.x)/len(data)/4))
print("rounded mean final value: \t {}".format(calc_diff(np.round(res.x, 3))/len(data)/4))
print("rounded final offset: {}".format(np.round(res.x, 3)))

Optimization terminated successfully.
         Current function value: 0.011492
         Iterations: 356
         Function evaluations: 589
initial value: 0.0149984257815
final value: 0.0114919559089
mean initial value: 	 	 0.000170436656608
mean final value: 	 	 0.000130590408056
rounded mean final value: 	 0.000130795410481
rounded final offset: [-0.    -0.003 -0.001 -0.001 -0.    -0.   ]


In [65]:
with robot:
    robot.set_joint_vel_level(1)

Connect: Server IP (192.168.21.2)


In [67]:
with robot:
    robot.joint_move_to([0]*6)

Connect: Server IP (192.168.21.2)


In [72]:
with robot:
    robot.joint_move_to(np.negative(np.rad2deg(res.x)))

Connect: Server IP (192.168.21.2)


In [73]:
with robot:
    print(np.round(robot.get_joint_pos(), 3))

Connect: Server IP (192.168.21.2)
[ 0.009  0.168  0.065  0.062 -0.     0.   ]


In [74]:
print(np.round(np.negative(np.rad2deg(res.x)), 3))

[0.009 0.168 0.065 0.062 0.    0.   ]


In [ ]:
Qnew = Q + dq